# APMTH 207: Advanced Scientific Computing: 
## Stochastic Methods for Data Analysis, Inference and Optimization
## Homework #5
**Harvard University**<br>
**Fall 2018**<br>
**Instructors: Rahul Dave**<br>
**Due Date: ** Saturday, October 13th, 2018 at 11:59pm

**Instructions:**

- Upload your final answers in the form of a Jupyter notebook containing all work to Canvas.

- Structure your notebook and your work to maximize readability.

### Collaborators

Jackie Chea, Dianne Lee, Grace Young

<div class="answer-separator">
------------------------
</div>

In [2]:
import numpy as np
import scipy.stats
import scipy.special

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import cm
import pandas as pd
%matplotlib inline

## Question 1: We'll Always Have that Night Sampling in Monte Carlo

**Coding required**


Let $X$ be a random variable with distribution described by the following pdf:

$$
f_X(x) = \begin{cases}
\frac{1}{12}(x-1), &1\leq x\leq 3\\
-\frac{1}{12}(x-5), &3< x\leq 5\\
\frac{1}{6}(x-5), &5< x\leq 7\\
-\frac{1}{6}(x-9), &7< x\leq 9\\
0, &otherwise
\end{cases}
$$

Let $h$ be the following function of $X$:

$$h(X) = \frac{1}{3\sqrt{2}\pi}\mathrm{exp}\left\{ -\frac{1}{18}\left( X - 5\right)^2\right\}$$


Compute $\mathbb{E}[h(X)]$ via Monte Carlo simulation using the following sampling methods:

**1.1.** Inverse Transform Sampling

**1.2.** Rejection Sampling with a uniform proposal distribution (rejection sampling in a rectangular box with uniform probability of sampling any x)


### 1.1 Inverse Transform Sampling


$$
y = F_X(x) = 
\begin{cases}
    \int_1^x \frac{1}{12}(t-1)dt = \frac{1}{24}x^2-\frac{1}{12}x+\frac{1}{24}, &1\leq x\leq 3\\
    F_X(3) + \int_3^x -\frac{1}{12}(t-5)dt = F_X(3) -\frac{1}{24}x^2+\frac{5}{12}x-\frac{21}{24}, &3< x\leq 5\\
    F_X(5) + \int_5^x \frac{1}{6}(t-5)dt = F_X(5) + \frac{1}{12}x^2-\frac{5}{6}x+\frac{25}{12}, &5< x\leq 7\\
    F_X(7) + \int_7^x -\frac{1}{6}(t-9)dt = F_X(7) -\frac{1}{12}x^2+\frac{3}{2}x-\frac{77}{12}, &7< x\leq 9\\
0, &otherwise
\end{cases}
$$
Solve for inverse:
$$
x = 
\begin{cases}
    2\sqrt{6y}+1, &0\leq y \leq \frac{1}{6}\\
    2\sqrt{1-6y}+5, &\frac{1}{6} < y \leq \frac{1}{3}\\
    2\sqrt{3y}+5, &\frac{1}{3} < y \leq \frac{2}{3}\\
    2\sqrt{1-3y}+9, &\frac{2}{3} < y \leq 1
\end{cases}
$$

In [4]:
y = np.linspace(0,1,20)
np.piecewise(y, 
            [0 <= y <= 1/6, 1/6 < y <= 1/3, 1/3 < y <= 2/3, 2/3 < y <= 1],
            [2*np.sqrt(6*y)+1, 2*np.sqrt(1-6*y)+5, 2*np.sqrt(3*y)+5, 2*np.sqrt(1-3*y)+9])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [19]:
# define inverse CDF
def invCDF (y):
    if 0 <= y <= 1/6:
        return 2*np.sqrt(6*y)+1
    elif 1/6 < y <= 1/3:
        return 2*np.sqrt(1-6*y)+5
    elif 1/3 < y <= 2/3:
        return 2*np.sqrt(3*y)+5
    elif 2/3 < y <= 1:
        return 2*np.sqrt(1-3*y)+9
    else:
        return 1

# domain limits
xmin = 0
xmax = 9

# range limits
ymin = 0
ymax = 1

N = 1000 # total of samples we wish to generate

# generate uniform samples in range, then invert CDF to get samples of target dist
Y = np.random.uniform(ymin, ymax, N)
X = invCDF(Y)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [14]:
invCDF(1/2)

7.4494897427831779

<div class="answer-separator">
------------------------
</div>

## Question 2: The Consequences of O-ring Failure can be Painful and Deadly

**Coding required**

In 1986, the space shuttle Challenger exploded during take off, killing the seven astronauts aboard. It is believed that the explosion was caused by the failure of an O-ring (a rubber ring that seals parts of the solid fuel rockets together), and that the failure was caused by the cold weather at the time of launch (31F).

In the file chall.txt, you will find temperature (in Fahrenheit) and failure data from 23 shuttle launches, where 1 stands for O-ring failure and 0 no failure. We assume that the observed temperatures are fixed and that, at temperature $t$, an O-ring fails with probability $f(\theta_{1}+\theta_{2}t)$ conditionally on $\Theta = (\theta_1, \theta_2)$.

$f(\mathbf{z})$ is defined to be the logistic function -- $f(\mathbf{ z }) = 1/(1 + \exp(\mathbf{ -z }))$ 

**2.1.** Based on your own knowledge and experience, suggest a prior distribution for the regression parameters ($\theta_1, \theta_2$).  Make sure to explain your choice of prior. 

**2.2.** Produce 5000-10000 samples from the posterior distribution of $\Theta $ using rejection sampling, and plot them and their marginals. (This may take a while.)

**2.3.** Use the logit package in the `statsmodels` library to compute 68% confidence intervals on the $\theta$ parameters.  Compare those intervals with the 68% credible intervals from the posterior above. Overlay these on the above marginals plots. 

**2.4.** Use the MLE values from `statsmodels` and the posterior mean from **2.2** at each temperature to plot the probability of failure in the frequentist and bayesian settings as a function of temperature. What do you see? 

**2.5.** Compute the mean posterior probability for an O-ring failure at $t = 31^\circ F$. To do this you must calculate the posterior at $31^\circ F$ and take the mean of the samples obtained.

**2.6.** You can instead obtain the probability from the posterior predictive. Use the posterior samples to obtain samples from the posterior predictive at $31^\circ F$ and calculate the fraction of failures.

**2.7.** The day before a new launch, meteorologists predict that the temperature will be $T \sim N(68, 1)$ during take-off. Estimate the probability for an O-ring failure during this take-off. (You will calculate multiple predictives at different temperatures for this purpose).

<div class="answer-separator">
------------------------
</div>

## Question 3: Maximum Uniformity -- Frequentist Bootstraps and the Bayesian Posterior

**Coding required**

Recall in HW 3 Question 1 we attempted to explore an edge case in using non-parametric bootstrap to construct confidence intervals.  Let's revisit the setup of that problem.
Suppose you have $\{X_1, X_2, ... X_n\}$ datapoints such that $X_i$ are independently and identically drawn from a $Unif(0, \theta)$.  Consider the extreme order statistic Y = $X_{(n)}$ = max($X_1, X_2, ... X_n$).

**3.1.** Derive (or possibly re-write from HW3) expressions for $F_Y(y\ \vert\ n, \theta)$ the CDF of Y and $f_Y(y\ |\ n, \theta)$ the pdf of Y.

**3.2.** In HW3 we had difficulty constructing confidence intervals to estimate $\theta$ using our normal percentiles methods so instead we introduced pivot confidence intervals.  Let's reframe the problem so that we can use percentiles to construct our confidence intervals.  Define $Z \equiv n \cdot (\theta - Y)$ and use elementary calculation to write an expression for $F_Z(z\ \vert\ n, \theta)$ the CDF of $Z$ and $f_Z(z\ \vert\ n, \theta)$ the pdf of Z.

**3.3.** What is the limiting distribution of Z (as $n \rightarrow \infty$)?  Plot that limiting distribution.

**3.4.** Use scipy/numpy to generate 100000 samples {$X_i$} from Unif(0,100) (i.e. let $\theta$ = 100).  Store them in Based on your data sample, what's $\hat{\theta}$ the empirical estimate for $\theta$.

**3.5.** Use non-parametric bootstrap to generate a sampling distribution of 10000 estimates for $Z$ by substituting $\hat{\theta}$ for $\theta$.  Plot a histogram of your sampling distribution.  Make sure to title and label the plot.  Use percentiles to construct the 10% and 68% bootstrap confidence intervals.  Plot them in your graph.

**Hint:  Should the confidence intervals be symmetric around the estimate $\hat{\theta}$**?

**3.6.** Make an argument that we can construct a bootstrap confidence interval that always mismatches the limiting distribution.

**3.7.** Let's switch to being Bayesian.  In 3.1 we came up with an expression for the likelihood $f_Y(y\ |\ n, \theta)$.  Use the [Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution) to construct a prior for $\theta$.  What are some reasonable values to use for the scale and shape?

**3.8.** Write down an expression for the posterior distribution $f_Y(\theta\ |\ n, y)$ 

**3.9.** Draw 10000 posterior samples and plot a histogram of the posterior distribution.  Use percentiles to construct the 68% HPD.  Plot the posterior distribution and mark the HPD on your plot.

**3.10.** How does the 68% HPD compare with the confidence interval generated from bootstrapping?  Why doesn't the bayesian interval construction suffer the same concerns you noted in 3.6

<div class="answer-separator">
------------------------
</div>